# Processing careless outputs

`/n/holyscratch01/hekstra_lab/hwang/monochromatic/dfs/process/process_careless_outputs.ipynb`

In this notebook, I process a few test drug fragment screening careless runs and summarize resultant statistics. I explore the effect of varying $r_{dw}$, `student-t-dof`, number of iterations, and number of holo datasets on the CCpred and the difference map peak heights, and I find that number of datasets doesn't appreciably affect these readouts. Additionally, for a dataset number equal to 4, I find that the optimal $r_{dw}$=0.9 and `student-t-dof`=128. 

Up until this point, I have checked the indexing ambiguity and flipped any datasets that have low pearson correlation with the reference. I manually reject outliers by throwing away reflections with intensity >2e5. the cleaned data are then input into `careless`. An example `careless` script can be found at `careless_first4-mlp_layers-rdw0,9_stdT.sh` in this folder. 

## I process the drug fragment screening `careless` runs as follows:

For each careless run, I run the processing cells once. At the end of this notebook, I provide a summary of some relevant statistics. 
1) I compute CChalf and CCpred using the `careless` functions, which is done in `stats_for_production.sh`. I compute both statistics as a function of resolution bin as well as an overall CChalf/CCpred. 
2) I rigid-body refine selected MTZ files against the apo_edit.pdb file, which is done in `autorefine.sh`. I usually RBR the `merged_{1,2,3}.mtz` files and sometimes the `{4,5}.mtz` files. For a general careless run, the numbering is:

| careless output number | description |
|------|--------|
| 0 | apo reference |
| 1 | apo experiment (negative control) |
| 2 | holo P0115 | 
| 3 | holo P0116 | 
| 4 | holo P0123 | 
| 5 | holo P0124 |


3) I compute difference maps of holo minus apo experiment using `rs.diffmap` with $\alpha=0.05$ weights and keeping only reflection with resolution in the 5-0.89 Å range. I measure difference peak heights using `rs.find_difference_peaks`, which is done in `make_diffmap.sh`. 
4) I summarize the statistics below.  

In [1]:
import os
import time
import pandas as pd
import glob
import numpy as np
import reciprocalspaceship as rs

In [2]:
os.environ["target_dir"]="careless_runs/merge_20103448_19373_mono_mc1_10k_grid_7"

In [7]:
rs.read_mtz("careless_runs/merge_20103448_19373_mono_mc1_10k_grid_7/dfs_0.mtz").spacegroup

<gemmi.SpaceGroup("P 43")>

In [8]:
%%bash
for dir in ${target_dir} ;do 
#for dir in out_test_first4-no_dw;do
    #sbatch make_diffmap-cut_1,5.sh ${dir}
    sbatch make_diffmap_norbr-noweights.sh ${dir}
done

Submitted batch job 20379183


In [4]:
%%bash
#out_test_first4-rdw0,8_stdT128 out_test_first4-rdw0,99_stdT128 
for dir in ${target_dir} ;do 
    mkdir ${dir}/phenix
    cd ${dir}/phenix
    cp ../../autorefine.sh .
    
    for dataset in {1..5};do

        #for the old datasets, like those from out_test_first20-in_order, we should use the flipped datasets .._flip.mtz. 
        sbatch scripts/autorefine.sh ../ $dataset merged_e35_${dataset}.mtz ../apo_edit_new.pdb
    done 
    cd ../../
done

cp: cannot stat '../../autorefine.sh': No such file or directory
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
cp: cannot stat '../../autorefine.sh': No such file or directory
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
cp: cannot stat '../../autorefine.sh': No such file or directory
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch: error: Unable to open file scripts/autorefine.sh
sbatch:

In [ ]:
time.sleep(240)

# CCpred  

In [9]:
%%bash
for file in ${target_dir};do
    sh scripts/stats_for_production.sh $file
done

/n/holyscratch01/hekstra_lab/hwang/monochromatic/dfs/process
/n/holyscratch01/hekstra_lab/hwang/monochromatic/dfs/process/out_test_first16-rdw0,936_stdT16-ohp-mlpw25-alf


Traceback (most recent call last):
  File "/net/holy-nfsisilon/ifs/rc_labs/hekstra_lab/people/hwang/anaconda/envs/careless/bin/careless.ccpred", line 8, in <module>
    sys.exit(main())
  File "/net/holy-nfsisilon/ifs/rc_labs/hekstra_lab/people/hwang/anaconda/envs/careless/lib/python3.10/site-packages/careless/stats/ccpred.py", line 141, in main
    run_analysis(parser)
  File "/net/holy-nfsisilon/ifs/rc_labs/hekstra_lab/people/hwang/anaconda/envs/careless/lib/python3.10/site-packages/careless/stats/ccpred.py", line 68, in run_analysis
    _ds = rs.read_mtz(m)
  File "/net/holy-nfsisilon/ifs/rc_labs/hekstra_lab/people/hwang/anaconda/envs/careless/lib/python3.10/site-packages/reciprocalspaceship/io/mtz.py", line 187, in read_mtz
    gemmi_mtz = gemmi.read_mtz_file(mtzfile)
FileNotFoundError: [Errno 2] Failed to open merged_e35_predictions_*.mtz: No such file or directory


# Summary statistics

## all datasets

In [4]:
file_list = ["careless_runs/merge_20103448_19373_mono_mc1_10k_grid_7"
                            
                           ]
# for i,file_in in enumerate(file_list):
#     os.system(f"sbatch scripts/make_diffmap_norbr-noweights.sh {file_in}")


In [8]:
print(f'    {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)
for i,file_in in enumerate(file_list):
    #extract peak heights of difference map peaks near the ligand in each model
    p0115_stats = pd.read_csv(f"{file_in}/out_P0115_noweights.csv")
    p0115_heights = p0115_stats[p0115_stats["residue"]=="LIG"]
    p0115_heights = p0115_heights["peakz"][:5].to_numpy().round(decimals=2)
    
    p0116_stats = pd.read_csv(f"{file_in}/out_P0116_noweights.csv")
    p0116_heights = p0116_stats[p0116_stats["residue"]=="LIG"]
    p0116_heights = p0116_heights["peakz"][:5].to_numpy().round(decimals=2)

    #extract the test CCpred of each holo mtz file
    bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")

    ccpred_p0115_condition = (bulk_ccpreds["file"]=="merged_e35_predictions_2.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0115 = bulk_ccpreds[ccpred_p0115_condition]["CCpred"]
    ccpred_p0116_condition = (bulk_ccpreds["file"]=="merged_e35_predictions_3.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0116 = float(bulk_ccpreds[ccpred_p0116_condition]["CCpred"].iloc[0])
    
    
    print(f'{i:2d}  {file_in:50s} {"P0115":10s} {float(bulk_ccpred_p0115.iloc[0]):0.4f}               {list(p0115_heights)}')
    print(f'{i:2d}  {file_in:50s} {"P0116":10s} {float(bulk_ccpred_p0116):0.4f}               {list(p0116_heights)}')
    print("-"*120)

    careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
 0  out_test_first8-rdw0,936_stdT16-ohp                P0115      0.9966               [36.2, -19.71, -17.75, -17.72, 15.87]
 0  out_test_first8-rdw0,936_stdT16-ohp                P0116      0.9874               [-13.83, 8.9, 7.96, -7.95, -7.72]
------------------------------------------------------------------------------------------------------------------------
 1  out_test_first8-rdw0,936_stdT16-ohp-mlpw           P0115      0.9967               [36.57, -19.85, -17.94, -17.34, 15.38]
 1  out_test_first8-rdw0,936_stdT16-ohp-mlpw           P0116      0.9877               [-14.61, 9.32, 8.77, -8.44, -7.93]
------------------------------------------------------------------------------------------------------------------------
 2  out_test_first4-rdw0,936_std

FileNotFoundError: [Errno 2] No such file or directory: 'out_test_first16-rdw0,936_stdT16-ohp-mlpw25-alf/e35_ccpred_b.csv'

In [ ]:
print(f'    {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)
for i,file_in in enumerate(file_list):
    #extract peak heights of difference map peaks near the ligand in each model
    p0115_stats = pd.read_csv(f"{file_in}/out_P0115-noweights.csv")
    p0115_heights = p0115_stats[p0115_stats["residue"]=="LIG"]
    p0115_heights = p0115_heights["peakz"][:5].to_numpy().round(decimals=2)
    
    p0116_stats = pd.read_csv(f"{file_in}/out_P0116-noweights.csv")
    p0116_heights = p0116_stats[p0116_stats["residue"]=="LIG"]
    p0116_heights = p0116_heights["peakz"][:5].to_numpy().round(decimals=2)

    #extract the test CCpred of each holo mtz file
    bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")

    ccpred_p0115_condition = (bulk_ccpreds["file"]=="merged_e35_predictions_2.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0115 = bulk_ccpreds[ccpred_p0115_condition]["CCpred"]
    ccpred_p0116_condition = (bulk_ccpreds["file"]=="merged_e35_predictions_3.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0116 = float(bulk_ccpreds[ccpred_p0116_condition]["CCpred"].iloc[0])
    
    
    print(f'{i:2d}  {file_in:50s} {"P0115":10s} {float(bulk_ccpred_p0115.iloc[0]):0.4f}               {list(p0115_heights)}')
    print(f'{i:2d}  {file_in:50s} {"P0116":10s} {float(bulk_ccpred_p0116):0.4f}               {list(p0116_heights)}')
    print("-"*120)

In [10]:
print(f'    {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)
for i,file_in in enumerate(["out_test_first4-all_metadata",
                            "out_test_first4-alf",
                            "out_test_first4-rdw0,936_stdT128-moreiters",
                            "out_test_first4-rdw-none_stdT16",
                            "out_test_first4-rdw0,0_stdT16",
                            "out_test_first4-rdw0,5_stdT16",
                            "out_test_first4-rdw0,75_stdT16",
                            "out_test_first4-rdw0,875_stdT16",
                            "out_test_first4-rdw0,936_stdT16",
                            "out_test_first4-rdw0,96875_stdT16",
                            "out_test_first4-rdw0,984375_stdT16",
                            "out_test_first4-rdw0,936_stdT8",
                            "out_test_first4-rdw0,936_stdT32",
                            "out_test_first4-rdw0,936_stdT64",
                            "out_test_first4-rdw0,936_stdT128",
                            "out_test_first4-rdw0,936_stdT256",
                            "out_test_first4-rdw0,936_stdT8-stacking",
                            "out_test_first4-rdw0,936_stdT32-alf",
                            "out_test_first4-rdw0,936_stdT64-alf",
                            "out_test_first4-rdw0,936_stdT128-alf",
                            "out_test_first4-rdw0,936_stdT256-alf",
                            "out_test_first3-rdw0,936_stdT16",
                            "out_test_first3-rdw0,936_stdT16-ohp",
                            "out_test_first4-rdw0,936_stdT16-file_id",
                            "out_test_first4-rdw0,936_stdT16-ohp",
                            "out_test_first8-rdw0,936_stdT16-ohp", 
                            "out_test_first16-rdw0,936_stdT16-ohp",
                            "out_test_first3-rdw0,936_stdT16-ohp-mlpw",
                            "out_test_first4-rdw0,936_stdT16-ohp-mlpw",
                            "out_test_first4-rdw0,936_stdT16-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT16-ohp-mlpw-alf",
                            "out_test_first8-rdw0,936_stdT16-ohp-mlpw",
                            "out_test_first3-rdw0,936_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT16-ohp-mlpw-alf", 
                            "out_test_first3-rdw0,936_stdT32-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT64-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT128-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT256-ohp-mlpw-alf",
                            "out_test_first3-rdwnone_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,5_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,75_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,875_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,96875_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,984375_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT16-ohp-mlpw15",
                            "out_test_first4-rdw0,936_stdT16-ohp-mlpw15",
                            "out_test_first8-rdw0,936_stdT16-ohp-mlpw",
                            
                           ]):
    
    #extract peak heights of difference map peaks near the ligand in each model
    p0115_stats = pd.read_csv(f"{file_in}/out_P0115.csv")
    p0115_heights = p0115_stats[p0115_stats["residue"]=="LIG"]
    p0115_heights = p0115_heights["peakz"][:5].to_numpy().round(decimals=2)
    
    p0116_stats = pd.read_csv(f"{file_in}/out_P0116.csv")
    p0116_heights = p0116_stats[p0116_stats["residue"]=="LIG"]
    p0116_heights = p0116_heights["peakz"][:5].to_numpy().round(decimals=2)

    #extract the test CCpred of each holo mtz file
    bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")

    ccpred_p0115_condition = (bulk_ccpreds["file"]=="merged_e35_predictions_2.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0115 = bulk_ccpreds[ccpred_p0115_condition]["CCpred"]
    ccpred_p0116_condition = (bulk_ccpreds["file"]=="merged_e35_predictions_3.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0116 = float(bulk_ccpreds[ccpred_p0116_condition]["CCpred"].iloc[0])
    
    
    print(f'{i:2d}  {file_in:50s} {"P0115":10s} {float(bulk_ccpred_p0115.iloc[0]):0.4f}               {list(p0115_heights)}')
    print(f'{i:2d}  {file_in:50s} {"P0116":10s} {float(bulk_ccpred_p0116):0.4f}               {list(p0116_heights)}')
    print("-"*120)


    careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
 0  out_test_first4-all_metadata                       P0115      0.9969               [30.0, -17.31, 14.96, -14.69, -14.19]
 0  out_test_first4-all_metadata                       P0116      0.9886               [-13.23, -8.11, 7.89, -7.83, 7.15]
------------------------------------------------------------------------------------------------------------------------
 1  out_test_first4-alf                                P0115      0.9966               [34.98, -21.04, -17.76, -17.71, 16.29]
 1  out_test_first4-alf                                P0116      0.9881               [-15.44, -9.43, 8.89, 8.85, -8.74]
------------------------------------------------------------------------------------------------------------------------
 2  out_test_first4-rdw0,936_st

## STD-T scans

In [ ]:
print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123","P0124"]):
    for i,file_in in enumerate([
                                "out_test_first4-rdw0,936_stdT8",
                                "out_test_first4-rdw0,936_stdT16",
                                "out_test_first4-rdw0,936_stdT32",
                                "out_test_first4-rdw0,936_stdT64",
                                "out_test_first4-rdw0,936_stdT128",
                                "out_test_first4-rdw0,936_stdT256",
                                "out_test_first4-rdw0,936_stdT32-alf",
                                "out_test_first4-rdw0,936_stdT64-alf",
                                "out_test_first4-rdw0,936_stdT128-alf",
                                "out_test_first4-rdw0,936_stdT256-alf",
                                "out_test_first3-rdw0,936_stdT8-ohp-mlpw-alf",
                                "out_test_first3-rdw0,936_stdT16-ohp-mlpw-alf", 
                                "out_test_first3-rdw0,936_stdT32-ohp-mlpw-alf",
                                "out_test_first3-rdw0,936_stdT64-ohp-mlpw-alf",
                                "out_test_first3-rdw0,936_stdT128-ohp-mlpw-alf",
                                "out_test_first3-rdw0,936_stdT256-ohp-mlpw-alf",
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        heights_sum = np.sum(np.abs(heights))
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i:2d}  {file_in:50s} {holo:10s} {float(bulk_ccpred.iloc[0]):0.4f}        {list(heights)}  {heights_sum:3.2f}')
    print("-"*120)

## iterations, keys, and misc

In [11]:
print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123","P0124"]):
    for i,file_in in enumerate(["out_test_first4-all_metadata",
                            "out_test_first4-alf",
                            "out_test_first4-rdw0,936_stdT128-moreiters",
                            "out_test_first4-rdw0,936_stdT16",
                            "out_test_first4-rdw0,936_stdT8-stacking",
                            "out_test_first4-rdw0,936_stdT16-file_id",
                            "out_test_first4-rdw0,936_stdT16-ohp",
                            "out_test_first8-rdw0,936_stdT16-ohp", 
                            "out_test_first16-rdw0,936_stdT16-ohp",
                            "out_test_first4-rdw0,936_stdT16-ohp-mlpw",
                            "out_test_first4-rdw0,936_stdT16-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT16",
                            "out_test_first3-rdw0,936_stdT16-ohp",
                            "out_test_first3-rdw0,936_stdT16-ohp-mlpw",
                            "out_test_first3-rdw0,936_stdT16-ohp-mlpw-alf",
                            "out_test_first8-rdw0,936_stdT16-ohp-mlpw",
                                "out_test_first4-rdw0,936_stdT16-ohp-mlpw15",
                                "out_test_first3-rdw0,936_stdT16-ohp-mlpw15",
                           ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        heights_sum = np.sum(np.abs(heights))
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i:2d}  {file_in:50s} {holo:10s} {float(bulk_ccpred.iloc[0]):0.4f}        {list(heights)}  {heights_sum:3.2f}')
    print("-"*120)

   careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
 0  out_test_first4-all_metadata                       P0115      0.9969        [30.0, -17.31, 14.96, -14.69, -14.19]  91.15
 1  out_test_first4-alf                                P0115      0.9966        [34.98, -21.04, -17.76, -17.71, 16.29]  107.78
 2  out_test_first4-rdw0,936_stdT128-moreiters         P0115      0.9966        [35.41, -20.92, -17.83, -17.56, 16.28]  108.00
 3  out_test_first4-rdw0,936_stdT16                    P0115      0.9966        [35.07, -20.57, -17.78, -17.19, 16.01]  106.62
 4  out_test_first4-rdw0,936_stdT8-stacking            P0115      0.9966        [35.57, -21.19, -18.23, -17.97, 16.28]  109.24
 5  out_test_first4-rdw0,936_stdT16-file_id            P0115      0.9966        [35.24, -20.72, -17.6, -17.58, 16.15]  107.29
 6  out_te

FileNotFoundError: [Errno 2] No such file or directory: 'out_test_first3-rdw0,936_stdT16/out_P0124.csv'

using one-hot encoding, I get worse performance as I increase the number of datasets. But if I add mlp width, I get better performance as I increase the number of datasets. Unfortunately, I can't keep going up on the number of datasets, otherwise I run out of memory. (what can I do on an A100 with 80 gig of memory? are we doing science or engineering? science, right? if engineering, I'd still do 3 datasets with as much mlp width as I can, because that's the important part. If we want to report on science, I'd try to see the max dataset number I can do with mlp-width 15, and then go from there. I'd also throw in ALF1,BET1 flags. 

# DW sweep

In [9]:

print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123"]):
    for i,file_in in enumerate(["out_test_first3-rdwnone_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,0_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,5_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,75_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,875_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,936_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,96875_stdT8-ohp-mlpw-alf",
                            "out_test_first3-rdw0,984375_stdT8-ohp-mlpw-alf",
                           ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        heights_sum = np.sum(np.abs(heights))
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i:2d}  {file_in:50s} {holo:10s} {float(bulk_ccpred.iloc[0]):0.4f}        {list(heights)}  {heights_sum:3.2f}')
    print("-"*120)

   careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
 0  out_test_first3-rdwnone_stdT8-ohp-mlpw-alf         P0115      0.9878        [34.14, -18.99, -17.57, -16.35, 15.69]  102.74
 1  out_test_first3-rdw0,0_stdT8-ohp-mlpw-alf          P0115      0.9883        [34.04, -19.04, -17.52, -16.47, 15.76]  102.83
 2  out_test_first3-rdw0,5_stdT8-ohp-mlpw-alf          P0115      0.9926        [35.26, -20.14, -18.42, -17.7, 15.59]  107.11
 3  out_test_first3-rdw0,75_stdT8-ohp-mlpw-alf         P0115      0.9954        [36.38, -20.92, -18.67, -18.36, 15.77]  110.10
 4  out_test_first3-rdw0,875_stdT8-ohp-mlpw-alf        P0115      0.9964        [36.58, -21.31, -18.59, -18.56, 16.03]  111.07
 5  out_test_first3-rdw0,936_stdT8-ohp-mlpw-alf        P0115      0.9967        [36.74, -21.69, -18.61, -18.6, 16.27]  111.91
 6  out_t

### more holo difference maps to determine the effect of iteration number

In [4]:
print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123","P0124"]):
    for i,file_in in enumerate([
                               "out_test_first4-mlp_layers-rdw0.9",
                               "out_test_first4-mlp_layers-rdw0.9-iterations",
                               ]):
    
        #extract peak heights of difference map peaks near the ligand in each model
        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        #extract the test CCpred of each holo mtz file
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')
    print("-"*120)

   careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
0  out_test_first4-mlp_layers-rdw0.9                  P0115      0.9966               [30.69, -15.75, 14.78, -14.33, -13.67]
1  out_test_first4-mlp_layers-rdw0.9-iterations       P0115      0.9966               [32.85, -16.43, -15.51, 14.95, -14.79]
------------------------------------------------------------------------------------------------------------------------
0  out_test_first4-mlp_layers-rdw0.9                  P0116      0.9873               [-14.04, 8.67, -8.07, 7.56, 7.46]
1  out_test_first4-mlp_layers-rdw0.9-iterations       P0116      0.9873               [-14.06, 8.48, -7.81, 7.43, 7.34]
------------------------------------------------------------------------------------------------------------------------
0  out_test_first4-mlp_layers-rdw0.9 

Comparing peak heights, it is still ambiguous whether increasing iteration number matters. Comparing CCpreds, maybe CCpred improves a little bit. It probably doesn't make a difference to use 10k iterations over 30k iterations.

### more holo difference maps to determine the effect of datasets number

In [5]:
print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123","P0124"]):
    for i,file_in in enumerate([
                               "out_test_first4-mlp_layers-rdw0.9",
                               "out_test_first20-in_order",
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')
    print("-"*120)

   careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
0  out_test_first4-mlp_layers-rdw0.9                  P0115      0.9966               [30.69, -15.75, 14.78, -14.33, -13.67]
1  out_test_first20-in_order                          P0115      0.9961               [30.88, -16.06, -14.58, 14.31, -13.86]
------------------------------------------------------------------------------------------------------------------------
0  out_test_first4-mlp_layers-rdw0.9                  P0116      0.9873               [-14.04, 8.67, -8.07, 7.56, 7.46]
1  out_test_first20-in_order                          P0116      0.9865               [-14.45, 8.46, -8.04, 7.85, 7.15]
------------------------------------------------------------------------------------------------------------------------
0  out_test_first4-mlp_layers-rdw0.9 

In [ ]:
out_test_last33-dw_r

In [13]:
print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123","P0124"]):
    for i,file_in in enumerate([
                               "out_test_first4-mlp_layers-rdw0.9",
                               "out_test_first4-add_rlp",
                            "out_test_first32-file_id",
                            "out_test_first32-in_order",
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')
    print("-"*120)

   careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
0  out_test_first4-mlp_layers-rdw0.9                  P0115      0.9966               [30.69, -15.75, 14.78, -14.33, -13.67]
1  out_test_first4-add_rlp                            P0115      0.9969               [25.95, 12.96, -11.47, 10.76, -10.49]
2  out_test_first32-file_id                           P0115      0.9961               [30.42, -15.64, -14.16, 13.97, -13.33]
3  out_test_first32-in_order                          P0115      0.9960               [30.54, -15.67, -14.19, 13.9, -13.46]
------------------------------------------------------------------------------------------------------------------------
0  out_test_first4-mlp_layers-rdw0.9                  P0116      0.9873               [-14.04, 8.67, -8.07, 7.56, 7.46]
1  out_test_first4-add_rlp    

Comparing peak heights, it is still ambiguous whether increasing iteration number matters. Comparing CCpreds, CCpred drops a little bit. It probably doesn't make a difference to use 10k iterations over 30k iterations.

I still haven't compared the effect of resolution cut on the peak heights. 

I sweep over $r_{dw}$ values at dof 128: here I compare $r_{dw}=0.8,0.9,0.936,0.97,0.99$. 

In [24]:
print(f'   {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)
for i,file_in in enumerate(["out_test_first4-rdw-none_stdT128",
                            "out_test_first4-rdw0_stdT128",
                            "out_test_first4-rdw0,5_stdT128",
                            "out_test_first4-rdw0,75_stdT128",
                            "out_test_first4-rdw0,8_stdT128",
                            "out_test_first4-rdw0,875_stdT128",
                            "out_test_first4-mlp_layers-rdw0.9_stdT",
                            "out_test_first4-rdw0,936_stdT128",
                            "out_test_first4-rdw0,97_stdT128",
                            "out_test_first4-rdw0,99_stdT128",
                           ]):
    i
    #extract peak heights of difference map peaks near the ligand in each model
    p0115_stats = pd.read_csv(f"{file_in}/out_P0115.csv")
    p0115_heights = p0115_stats[p0115_stats["residue"]=="LIG"]
    p0115_heights = p0115_heights["peakz"][:5].to_numpy().round(decimals=2)
    
    p0116_stats = pd.read_csv(f"{file_in}/out_P0116.csv")
    p0116_heights = p0116_stats[p0116_stats["residue"]=="LIG"]
    p0116_heights = p0116_heights["peakz"][:5].to_numpy().round(decimals=2)

    #extract the test CCpred of each holo mtz file
    bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
    
    if file_in == "out_test_first4-no_dw":
        pnum=1
    else:
        pnum=2 
        
    ccpred_p0115_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{pnum}.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0115 = float(bulk_ccpreds[ccpred_p0115_condition]["CCpred"].iloc[0])
    ccpred_p0116_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{pnum+1}.mtz"
                             )*(bulk_ccpreds["test"]=="Test")
    bulk_ccpred_p0116 = float(bulk_ccpreds[ccpred_p0116_condition]["CCpred"].iloc[0])
    
    
    print(f'{i}  {file_in:50s} {"P0115":10s} {float(bulk_ccpred_p0115):0.4f}               {list(p0115_heights)}')
    print(f'{i}  {file_in:50s} {"P0116":10s} {float(bulk_ccpred_p0116):0.4f}               {list(p0116_heights)}')
    print("-"*120)


   careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
0  out_test_first4-rdw-none_stdT128                   P0115      0.9909               [32.59, -18.07, -15.29, 15.03, 14.71]
0  out_test_first4-rdw-none_stdT128                   P0116      0.9855               [-14.42, -9.15, 8.41, 7.91, 7.22]
------------------------------------------------------------------------------------------------------------------------
1  out_test_first4-rdw0_stdT128                       P0115      0.9960               [10.9, -8.7, 8.22, -7.65, 6.54]
1  out_test_first4-rdw0_stdT128                       P0116      0.9863               []
------------------------------------------------------------------------------------------------------------------------
2  out_test_first4-rdw0,5_stdT128                     P0115      0.9964     

Based on CCpred,test and the ligand peak rmsds, I think $r_{dw}=0.9$, `dof` 128 is still the best. 

Unlike the lysozyme anomalous or PDZ2 EFX datasets, the optimization landscape doesn't seem as rocky here. I am satisfied with the current results. I would be happy to provide a $r_{dw}$ sweep over a sensible range. 

Given the ok from Doeke, I will keep the $r_{dw}=0.9$, `dof` 128 parameters and scale up to a 32-dataset run and a 33-dataset run (I already checked that these will run on an NVIDIA v100 GPU with 32GB memory).

# Difference maps around the ligand

I visualize difference maps in coot. This is from the `out_test_first-20-in_order` `careless` run. 

P0115

<div class="row">
  <div class="column">
    <img src="P0115.png" alt="drawing" width="400"/>
  </div>
  <div class="column">
    <img src="P0116.png" alt="drawing" width="400"/>
  </div>
</div>

P0116

Now let's process a ton of datasets simultaneously. 

In [ ]:
%%bash
for dir in out_test_first32-in_order;do 
#for dir in out_test_first32-in_order out_test_last33;do 
    mkdir ${dir}/phenix
    cd ${dir}/phenix
    cp ../../autorefine.sh .
    
    for dataset in {1..34};do

        #for the old datasets, like those from out_test_first20-in_order, we should use the flipped datasets .._flip.mtz. 
        sbatch autorefine.sh ../ $dataset merged_e35_${dataset}.mtz ../apo_edit.pdb
    done 
    cd ../../
done

Submitted batch job 13004643
Submitted batch job 13004644
Submitted batch job 13004645
Submitted batch job 13004646
Submitted batch job 13004647
Submitted batch job 13004648
Submitted batch job 13004649
Submitted batch job 13004650
Submitted batch job 13004651
Submitted batch job 13004652
Submitted batch job 13004653
Submitted batch job 13004654
Submitted batch job 13004655
Submitted batch job 13004656
Submitted batch job 13004657
Submitted batch job 13004658
Submitted batch job 13004659
Submitted batch job 13004660
Submitted batch job 13004661
Submitted batch job 13004662
Submitted batch job 13004663
Submitted batch job 13004664
Submitted batch job 13004665
Submitted batch job 13004666
Submitted batch job 13004667
Submitted batch job 13004668
Submitted batch job 13004669
Submitted batch job 13004670
Submitted batch job 13004671
Submitted batch job 13004672
Submitted batch job 13004673
Submitted batch job 13004674
Submitted batch job 13004675
Submitted batch job 13004676


In [52]:
pwd

'/n/holyscratch01/hekstra_lab/hwang/monochromatic/dfs/process'

In [57]:
%%bash

for dir in out_test_first32-in_order out_test_last33;do 
    sh stats_for_production.sh ${dir}
done

/n/home01/hwang6/mambaforge/envs/tf2.12_cuda11/lib/python3.10/site-packages/careless/stats/ccpred.py:131: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()
/n/home01/hwang6/mambaforge/envs/tf2.12_cuda11/lib/python3.10/site-packages/careless/stats/ccpred.py:131: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [15]:
%%bash
for dir in out_test_first32-in_order ;do 
    #sbatch make_diffmap-first32.sh ${dir}
    sbatch make_diffmap-first32-cut_1,5.sh ${dir}
done

for dir in out_test_last33-dw_r;do 
    #sbatch make_diffmap-last33.sh ${dir}
    sbatch make_diffmap-last33-cut_1,5.sh ${dir}
done

Submitted batch job 13326988
Submitted batch job 13326989


In [3]:
%%bash
for dir in out_test_first32-in_order ;do 
    sbatch make_diffmap-first32.sh ${dir}
done

# for dir in out_test_last33-no_dw ;do 
#     sbatch make_diffmap-last33.sh ${dir}
# done

Submitted batch job 14727984


In [2]:
%%bash
scancel 14727066

In [ ]:
a = rs.read_mtz("out_test_first32-in_order/dfs_49.mtz")
b = rs.read_mtZ("

In [18]:
file_pattern = '../20221007_unscaled_unmerged/UCSF-P*/out.mtz'
files = glob.glob(file_pattern)
dataset_names = []
files.sort()
for name in files:
    dataset_names.append(name[-13:-8])

In [56]:
print(f'    {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)
for j,holo in enumerate(dataset_names[:32]):
    for i,file_in in enumerate([
                               "out_test_first32-in_order",
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{j+2:2.0f}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')
        
for j,holo in enumerate(dataset_names[32:]):
    for i,file_in in enumerate([
                               "out_test_last33",
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{j+2:2.0f}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')

    careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
 2  out_test_first32-in_order                          P0115      0.9960               [30.54, -15.67, -14.19, 13.9, -13.46]
 3  out_test_first32-in_order                          P0116      0.9875               [-14.16, 8.86, -7.93, 7.84, 7.37]
 4  out_test_first32-in_order                          P0123      0.9974               [-7.78, 6.99, 6.79, 6.38, -6.09]
 5  out_test_first32-in_order                          P0124      0.9968               [-16.64, 11.83, 11.47, 10.58, -9.62]
 6  out_test_first32-in_order                          P0131      0.9964               [-13.85, 10.86, -9.67, -9.29, 8.24]
 7  out_test_first32-in_order                          P0132      0.9950               [-15.09, 12.28, 12.09, -9.44, 9.25]
 8  out_test_first32-in_order   

Does adding the file_id flag improve things? does dropping the dw flags do anything?

In [17]:
print(f'    {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)

for j,holo in enumerate(["P0115","P0116","P0123","P0124"]):
    for i,file_in in enumerate([
                            "out_test_first32-file_id",
                            "out_test_first32-in_order",
                            "out_test_first32-no_dw",
                            "out_test_first32-0",
                            "out_test_first32-0,5",
                            "out_test_first32-0,75",
                            "out_test_first32-0,875",
                            "out_test_first32-0,9375",
                            "out_test_first32-0,96875",
                            "out_test_first32-0,984375",
                            "out_test_first4-mlp_layers-rdw0.9_stdT"
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{i:2.0f}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')
    print("-"*120)

    careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________
 0  out_test_first32-file_id                           P0115      0.9961               [30.42, -15.64, -14.16, 13.97, -13.33]
 1  out_test_first32-in_order                          P0115      0.9960               [30.54, -15.67, -14.19, 13.9, -13.46]
 2  out_test_first32-no_dw                             P0115      0.9957               [29.58, -14.52, -13.82, 13.34, 12.86]
 3  out_test_first32-0                                 P0115      0.9957               [29.62, -14.53, -13.81, 13.32, 12.9]
 4  out_test_first32-0,5                               P0115      0.9959               [30.64, -15.31, -14.26, 13.85, -13.53]
 5  out_test_first32-0,75                              P0115      0.9960               [30.83, -15.57, -14.24, 13.81, -13.53]
 6  out_test_fir

In [13]:
import reciprocalspaceship as rs

Not particularly.

In [4]:
print(f'    {"careless run":50s} {"dataset":10s} {"CCpred,test":20s} {"top 5 ligand peak rmsds"}')
print("_"*120)
        
for j,holo in enumerate(dataset_names[32:]):
    for i,file_in in enumerate([
                               "out_test_last33-dw_r",
                               ]):
    

        stats = pd.read_csv(f"{file_in}/out_{holo}.csv")
        heights = stats[stats["residue"]=="LIG"]
        heights = heights["peakz"][:5].to_numpy().round(decimals=2)
        
        bulk_ccpreds = pd.read_csv(f"{file_in}/e35_ccpred_b.csv")
        ccpred_condition = (bulk_ccpreds["file"]==f"merged_e35_predictions_{j+2}.mtz"
                                 )*(bulk_ccpreds["test"]=="Test")
        bulk_ccpred = bulk_ccpreds[ccpred_condition]["CCpred"]
        
        print(f'{j+2:2.0f}  {file_in:50s} {holo:10s} {float(bulk_ccpred):0.4f}               {list(heights)}')

    careless run                                       dataset    CCpred,test          top 5 ligand peak rmsds
________________________________________________________________________________________________________________________


NameError: name 'dataset_names' is not defined

In [64]:
a = """0.978,0.979,0.981,0.982,0.972,0.973,0.981,0.978,0.980,0.983,0.981,0.968,0.980,0.983,0.973,0.966,0.985,0.978,0.976,0.983,0.977,0.974,0.975,0.982,0.974,0.965,0.979,0.975,0.967,0.970,0.978,0.976,0.957,0.978,0.942,0.981,0.977,0.975,0.976,0.973,0.980,0.981,"""
incr = 0
for i,char in enumerate(a):
    if char==",":
        incr+=1
        if incr == 33:
            print(i)
        

197


In [65]:
a[:197]

'0.978,0.979,0.981,0.982,0.972,0.973,0.981,0.978,0.980,0.983,0.981,0.968,0.980,0.983,0.973,0.966,0.985,0.978,0.976,0.983,0.977,0.974,0.975,0.982,0.974,0.965,0.979,0.975,0.967,0.970,0.978,0.976,0.957'